# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** W

**Names:**

* Olivier Cloux
* Thibault Urien
* Saskia Reiss

---

#### Instructions

*This is a template for part 3 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---

## 2.3 Epdemics


#### Exercise 2.9: Simulate an epidemic outbreak

In [ ]:
# Imports
import epidemics_helper
import json
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from networkx.readwrite import json_graph


In [ ]:
# Instead of using requests, we use the json reader in networkx to be able to read and draw the graph.
with open('../data/nyc_augmented_network.json') as data_file:
    data = json.load(data_file)
    g = json_graph.node_link_graph(data)
    sir = epidemics_helper.SimulationSIR(g, beta=10.0, gamma=0.1)
    sir.launch_epidemic(23654,100)

In [ ]:
def plotSIR(simu):
    # Plot the susceptible people over time
    sortedS = list(sorted(map( lambda x : min(x, simu.max_time),simu.inf_time)))
    valuesS, baseS = np.histogram(sortedS, bins=500)

    # Plot the infected people over time
    sortedI = list(filter(lambda x : x < simu.max_time,sortedS))
    valuesI, baseI = np.histogram(sortedI, bins=500,range=[0,100])

    # Plot the recovered people over time
    sortedR = list(sorted(filter( lambda x : x < simu.max_time,simu.rec_time)))
    valuesR, baseR = np.histogram(sortedR, bins=500,range=[0,100])

    #evaluate the cumulative, to be able to have curves instead of bins.
    cumulativeS = np.cumsum(valuesS)
    cumulativeR = np.cumsum(valuesR)
    cumulativeI = np.cumsum(valuesI) - cumulativeR

    # plot the cumulative susceptible function
    plt.plot(baseS[:-1], (len(sortedS)-cumulativeS)/len(sortedS) *100, c='blue', label="susceptible")
    #plot the cumulative infected function
    plt.plot(baseI[:-1],(cumulativeI )/len(sortedS) *100, c='g',label="infected")
    #plot the cumulative recovered function
    plt.plot(baseR[:-1], (cumulativeR)/len(sortedS) *100, c='red',label="recovered")

    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel("Time in days")
    plt.ylabel("Percentage of the population")
    plt.ylim([0, 100])
    plt.xlim([0, simu.max_time])
    plt.show();
    
plotSIR(sir)

In [ ]:
# Helper function to give each node the right color
def nodesColor(time, simu, colors = ['blue','green','red']):
    return list(map(lambda i,r : colors[0] if time < i else (colors[1] if time < r else colors[2]),simu.inf_time,simu.rec_time))
    

In [ ]:
def nyMap(time,G,simu):
    plt.figure(figsize=(20,20)) 
    plt.axis("off")
    positions = nx.get_node_attributes(G,'coordinates')
    nodes = nx.draw_networkx_nodes(G,positions,node_size=10)
    nodes.set_linewidth(0)
    colors = nodesColor(time,simu)
    nodes.set_color(colors)
    nx.draw_networkx_edges(G,positions)
    plt.title("State of epidemy at day "+str(time),{"fontsize":25})
    plt.show();

In [ ]:
nyMap(1,g,sir)
nyMap(3,g,sir)
nyMap(30,g,sir)

---

### 2.3.1 Stop the apocalypse!

#### Exercise 2.10: Strategy 1

In [ ]:
import random

def randomDelete(max, starter_node, graph):
    to_delete = []
    i = 0
    #need a copy to be able to do this several times withoutchanging g.
    G = graph.copy()
    # This will give us a list of all edges in the type [(node1, node2), ...]
    list_edges = G.edges()
    for i in range(0, max):
        # Choose a random number between 0 and the length of our array
        x = random.randint(0, len(list_edges)-1)
        # Remove this random edge
        to_delete.append(list_edges.pop(x))
    G.remove_edges_from(to_delete)
    sir = epidemics_helper.SimulationSIR(G, beta=10.0, gamma=0.1)
    # Launch the simulation to see how it is after 30 days.
    sir.launch_epidemic(starter_node, 30)


In [ ]:
print("Normal Simulation 1:")
sir = epidemics_helper.SimulationSIR(g, beta=10.0, gamma=0.1)
# We will choose a random starting node and use the same for both simulations (the "normal" and the random one)
starter_node = random.sample(g.nodes(), 1).pop()
# Launch the simulation to see how it is after 30 days.
sir.launch_epidemic(starter_node,30)
print("Random delete 1:")
randomDelete(1000, starter_node, g)

starter_node = random.sample(g.nodes(), 1).pop()
print("Normal Simulation 2:")
sir = epidemics_helper.SimulationSIR(g, beta=10.0, gamma=0.1)
# Launch the simulation to see how it is after 30 days.
sir.launch_epidemic(starter_node,30)
print("Random delete 2:")
randomDelete(1000, starter_node, g)

print("Normal Simulation 3:")
sir = epidemics_helper.SimulationSIR(g, beta=10.0, gamma=0.1)
# Launch the simulation to see how it is after 30 days.
sir.launch_epidemic(starter_node,30)
print("Random delete 3:")
randomDelete(1000, starter_node, g)


print("Random delete With 10000 deletions:")
randomDelete(10000, starter_node, g)

###### Conclusions
From these results, we see that results vary greatly when it is random, even if it is a bit more effective by having a bitter budget. We still see a very high indected rate, and maybe it does slow down the epidemy a little bit, but it is not as effective as it could be.


#### Exercise 2.11: Strategy 2

In [ ]:
def imunizePreSimu(G,beta,gamma, budget):
    threshold = 0.5
    starters = 10
    sir = epidemics_helper.SimulationSIR(g, beta=beta, gamma=gamma)
    startersIndex = random.sample(G.nodes(),starters)
    intresting_edges = set()
    for starter in startersIndex:
        
        start = starter
        for i in range(0 ,1000):
            start = random.sample(G.neighbors(start),1).pop()
        sir.launch_epidemic(starter,50)
        sortedS = list(sorted(map( lambda x : min(x, sir.max_time),sir.inf_time)))
        valuesS, baseS = np.histogram(sortedS, bins=500)
        cumulativeS = np.cumsum(valuesS)
        cutIndex = np.where(cumulativeS/cumulativeS[-1] > threshold )[0][0]
        cut = baseS[cutIndex]
        def egde_Is_intresting(edge):
            return (sir.inf_time[edge[0]] >= cut) &  (sir.inf_time[edge[1]] < cut) | (sir.inf_time[edge[1]] >= cut) &  (sir.inf_time[edge[0]] < cut)

        intresting_edges =  intresting_edges.union(filter(egde_Is_intresting,G.edges()))
    toBeRemoved = random.sample(intresting_edges,budget)
    returned = G.copy()
    returned.remove_edges_from(toBeRemoved)
    return returned

    

In [ ]:
def imunizeDelLongEges(G,beta,gamma, budget):
    positions = nx.get_node_attributes(G,'coordinates')
    def dist(edge):
        p1 = positions[edge[0]]
        p2 = positions[edge[1]]
        dx = p1[0]-p2[0]
        dy = p1[1]-p2[1]
        return dx**2 + dy **2
    dists = list(map(dist,G.edges()))
    longest_idx = sorted(range(len(dists)), key=lambda i: dists[i])[-budget:]
    print(len(longest_idx))
    longest = map(lambda x : G.edges()[x],longest_idx)
    returned = G.copy()
    returned.remove_edges_from(longest)
    return returned
    

In [ ]:
def imunizeNonBypassable(G,beta,gamma, budget):
    maxDepth = 5
    def dfs(n, target,depth = 0, visited=None):
        if (depth >= maxDepth):
            return maxDepth
        elif (n == target):
            if(depth == 1):
                return maxDepth
            else:
                return depth
        else:
            if visited is None:
                visited = set()
            visited.add(n)
            ret = maxDepth
            for nxt in set(G.neighbors(n)) - visited:
                depth = dfs(nxt,target,depth+1, visited)
                ret = min(ret,depth)
            return ret
    def bypassDist(edge):
        p1 = edge[0]
        p2 = edge[1]
        return min(dfs(p1,p2),dfs(p2,p1))
    
    dists = list(map(bypassDist,G.edges()))
    longest_idx = filter(lambda x : x == maxDepth,dists)
    longest = list(map(lambda x : G.edges()[x],longest_idx))
    returned = G.copy()
    returned.remove_edges_from(random.sample(longest,budget))
    return returned 
      
    

In [ ]:
imunizedG = imunizeNonBypassable(g,10.0,0.1,2500)
starter_node = random.sample(imunizedG.nodes(), 1).pop()
simu2 = epidemics_helper.SimulationSIR(imunizedG, beta=10.0, gamma=0.1)
simu2.launch_epidemic(starter_node,100)
nyMap(30,imunizedG,simu2)